<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Predicting shots made per game by Kobe Bryant


---

In this lab you'll be using regularized regression penalties Ridge, Lasso, and Elastic Net to try and predict how many shots Kobe Bryant made per game in his career.

The Kobe shots dataset has hundreds of columns representing different characteristics of each basketball game. Fitting an ordinary linear regression using every predictor would dramatically overfit the model considering the limited number of observations (games) we have available. Furthermore, many of the predictors have significant multicollinearity. 

**Warning:** Some of these calculations are computationally expensive and may take a while to execute.  It may be worth while to only use a portion of the data to perform these calculations, especially if you have experienced kernel issues in the past.

### 1. Load packages and data

In [1]:
import numpy as np
import pandas as pd
import patsy

from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import r2_score

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

In [2]:
kobe = pd.read_csv('../../../../resource-datasets/kobe_bryant/kobe_superwide_games.csv')

### 2. Examine the data

- How many columns are there?
- Examine what the observations (rows) and columns represent.
- Why might regularization be particularly useful for this dataset?

In [7]:
# A:
kobe.head()

,SHOTS_MADE,AWAY_GAME,SEASON_OPPONENT:atl:1996-97,SEASON_OPPONENT:atl:1997-98,SEASON_OPPONENT:atl:1999-00,SEASON_OPPONENT:atl:2000-01,SEASON_OPPONENT:atl:2001-02,SEASON_OPPONENT:atl:2002-03,SEASON_OPPONENT:atl:2003-04,SEASON_OPPONENT:atl:2004-05,...,ACTION_TYPE:tip_layup_shot,ACTION_TYPE:tip_shot,ACTION_TYPE:turnaround_bank_shot,ACTION_TYPE:turnaround_fadeaway_bank_jump_shot,ACTION_TYPE:turnaround_fadeaway_shot,ACTION_TYPE:turnaround_finger_roll_shot,ACTION_TYPE:turnaround_hook_shot,ACTION_TYPE:turnaround_jump_shot,SEASON_GAME_NUMBER,CAREER_GAME_NUMBER
0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,2
2,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,3
3,2.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,4
4,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,5


In [10]:
kobe.columns
# Regularization useful here as there are so many columns in the dataset, and there will inevitably be collinearity

Index(['SHOTS_MADE', 'AWAY_GAME', 'SEASON_OPPONENT:atl:1996-97',
       'SEASON_OPPONENT:atl:1997-98', 'SEASON_OPPONENT:atl:1999-00',
       'SEASON_OPPONENT:atl:2000-01', 'SEASON_OPPONENT:atl:2001-02',
       'SEASON_OPPONENT:atl:2002-03', 'SEASON_OPPONENT:atl:2003-04',
       'SEASON_OPPONENT:atl:2004-05',
       ...
       'ACTION_TYPE:tip_layup_shot', 'ACTION_TYPE:tip_shot',
       'ACTION_TYPE:turnaround_bank_shot',
       'ACTION_TYPE:turnaround_fadeaway_bank_jump_shot',
       'ACTION_TYPE:turnaround_fadeaway_shot',
       'ACTION_TYPE:turnaround_finger_roll_shot',
       'ACTION_TYPE:turnaround_hook_shot', 'ACTION_TYPE:turnaround_jump_shot',
       'SEASON_GAME_NUMBER', 'CAREER_GAME_NUMBER'],
      dtype='object', length=645)

### 3. Make predictor and target variables. Standardize the predictors.

Why is normalization necessary for regularized regressions?

Use the `sklearn.preprocessing` class `StandardScaler` to standardize the predictors.

In [11]:
# A: Normalization necessary as to scale the beta coefficience w.r.t each other appropriately, rather than diff scale.

# Make predictor and target variables:
y = kobe.pop('SHOTS_MADE')
X = kobe

# Standardize predictor:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
Xstd = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

### 4. Build a linear regression predicting `SHOTS_MADE` from the rest of the columns.

Cross-validate the $R^2$ of an ordinary linear regression model with 10 cross-validation folds.

How does it perform?

In [50]:
# A: Build the linear regression for y based on standardized X:
lr = LinearRegression()
lr.fit(Xstd, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [53]:
lr_df = pd.DataFrame(lr.coef_,columns=['lr_coef'],index=kobe.columns)
lr_df

,lr_coef
AWAY_GAME,-1.537869e-01
SEASON_OPPONENT:atl:1996-97,4.108329e+12
SEASON_OPPONENT:atl:1997-98,-8.554138e+11
SEASON_OPPONENT:atl:1999-00,6.380127e+11
SEASON_OPPONENT:atl:2000-01,-1.752928e+12
SEASON_OPPONENT:atl:2001-02,5.096673e+12
SEASON_OPPONENT:atl:2002-03,6.438449e+12
SEASON_OPPONENT:atl:2003-04,1.133049e+12
SEASON_OPPONENT:atl:2004-05,-9.746039e+12
SEASON_OPPONENT:atl:2005-06,-4.080974e+12


In [14]:
# cross-validation with 10-folds:
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn import metrics

kf = KFold(n_splits=10, shuffle=True, random_state=1)

scores = cross_val_score(lr, Xstd, y, cv=kf)
print("Cross-validated scores:", scores)
print("Mean of Cross-validated scores:", scores.mean())
print("Std of Cross-validated scores:", scores.std())

# very large .std() of the cv scores shows bad OLS performance

Cross-validated scores: [-7.80941423e+26 -4.26429154e+27 -9.94633064e+26 -2.37731098e+26
 -3.62724679e+26 -1.06648923e+27 -5.65054028e+25 -5.11655257e+27
 -5.22093219e+27 -1.24935857e+26]
Mean of Cross-validated scores: -1.8225737068120948e+27
Std of Cross-validated scores: 2.0333993025199608e+27


### 5. Find an optimal value for Ridge regression alpha using `RidgeCV`.

[Go to the documentation and read how RidgeCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html)

> *Hint: once the RidgeCV is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Ridge performs best searching alphas through logarithmic space (`np.logspace`). This may take awhile to fit!


In [56]:
# A:
ridge = RidgeCV(alphas=np.logspace(-5, 5, 100), cv=kf)
ridge.fit(X, y)
print(ridge.alpha_)

11.497569953977356


In [57]:
ridge_df = pd.DataFrame(ridge.coef_,columns=['ridge_coef'],index=kobe.columns)
ridge_df

,ridge_coef
AWAY_GAME,-0.221689
SEASON_OPPONENT:atl:1996-97,-0.113330
SEASON_OPPONENT:atl:1997-98,0.044487
SEASON_OPPONENT:atl:1999-00,0.009979
SEASON_OPPONENT:atl:2000-01,-0.051925
SEASON_OPPONENT:atl:2001-02,0.281550
SEASON_OPPONENT:atl:2002-03,-0.010784
SEASON_OPPONENT:atl:2003-04,0.102819
SEASON_OPPONENT:atl:2004-05,0.073769
SEASON_OPPONENT:atl:2005-06,-0.025472


### 6. Cross-validate the Ridge regression $R^2$ with the optimal alpha.

Is it better than the Linear regression? If so, why might this be?

In [16]:
# A:
from sklearn.linear_model import Ridge
ridge = Ridge(model.alpha_)
cv_score = (np.mean(cross_val_score(ridge, X, y, cv=kf)))
cv_score

0.6502145592846625

### 7. Find an optimal value for Lasso regression alpha using `LassoCV`.

[Go to the documentation and read how LassoCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) It is very similar to `RidgeCV`.

> *Hint: again, once the `LassoCV` is fit, the attribute `.alpha_` contains the best alpha parameter it found through cross-validation.*

Recall that Lasso, unlike Ridge, performs best searching for alpha through linear space (`np.linspace`). However, you can actually let the LassoCV decide itself what alphas to use by instead setting the keyword argument `n_alphas=` to however many alphas you want it to search over. It is recommended to let sklearn choose the range of alphas.

_**Tip:** If you find your CV taking a long time and you're not sure if its working set `verbose =1`._

In [58]:
# A:
lasso = LassoCV(alphas=np.linspace(0, 1, 100), cv=kf, max_iter=10000)
lasso.fit(X, y)
print(lasso.alpha_)

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1517.0923558103314, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1560.3741190122819, tolerance: 1.713358416547788

0.010101010101010102


In [59]:
lasso_df = pd.DataFrame(lasso.coef_,columns=['lasso_coef'],index=kobe.columns)
lasso_df

,lasso_coef
AWAY_GAME,-0.169962
SEASON_OPPONENT:atl:1996-97,-0.000000
SEASON_OPPONENT:atl:1997-98,0.000000
SEASON_OPPONENT:atl:1999-00,0.000000
SEASON_OPPONENT:atl:2000-01,-0.000000
SEASON_OPPONENT:atl:2001-02,0.000000
SEASON_OPPONENT:atl:2002-03,-0.000000
SEASON_OPPONENT:atl:2003-04,0.000000
SEASON_OPPONENT:atl:2004-05,0.000000
SEASON_OPPONENT:atl:2005-06,-0.000000


### 8. Cross-validate the Lasso $R^2$ with the optimal alpha.

Is it better than the Linear regression? Is it better than Ridge? What do the differences in results imply about the issues with the dataset?

In [29]:
# A:
from sklearn.linear_model import Lasso

lasso = Lasso(lasso.alpha_, max_iter=10000)
cv_score = (np.mean(cross_val_score(lasso, X, y, cv=kf)))
cv_score

0.6480641922345887

### 9. Look at the coefficients for variables in the Lasso.

1. Show the coefficient for variables, ordered from largest to smallest coefficient by absolute value.
2. What percent of the variables in the original dataset are "zeroed-out" by the lasso?
3. What are the most important predictors for how many shots Kobe made in a game?

> **Note:** if you only fit the Lasso within `cross_val_score`, you will have to refit it outside of that
function to pull out the coefficients.

In [43]:
# A: Refitting as it was most recently fit inside the cvs:

model = LassoCV(alphas=np.linspace(0, 1, 100), cv=kf, max_iter=10000)
model.fit(X, y)

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1517.0923558103314, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: UserWarning: Coordinate descent with alpha=0 may lead to unexpected results and is discouraged.
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1560.3741190122819, tolerance: 1.713358416547788

TypeError: Index(...) must be called with a collection of some kind, 'lasso_coef' was passed

In [54]:
lasso_df = pd.DataFrame(model.coef_, columns=['lasso_coef'])

In [55]:
lasso_df.sort_values('lasso_coef',ascending=False)

,lasso_coef
423,0.796920
577,0.337339
395,0.328992
566,0.279193
576,0.211404
541,0.195215
556,0.180891
579,0.150916
558,0.140908
543,0.106977


In [64]:
# create a nice df of all the model coeffs
coeffs = pd.concat([lr_df,ridge_df,lasso_df],axis=1)
coeffs.sort_values('lasso_coef',ascending=False)

# you see many more zero coefficients in the lasso

,lr_coef,ridge_coef,lasso_coef
SEASON_OPPONENT:sac:1999-00,1.909114e+12,1.125281,0.796920
COMBINED_SHOT_TYPE:dunk,1.216708e+12,0.261939,0.337339
SEASON_OPPONENT:phx:2009-10,4.215080e+12,0.811825,0.328992
SHOT_ZONE_BASIC:restricted_area,6.968329e+11,0.322359,0.279193
COMBINED_SHOT_TYPE:bank_shot,4.003756e+11,0.253613,0.211404
SEASON:2007-08,-5.765743e+12,0.418990,0.195215
SHOT_ZONE_RANGE:16-24_ft.,-5.430363e+12,0.189120,0.180891
COMBINED_SHOT_TYPE:jump_shot,7.535605e+12,0.124218,0.150916
SHOT_ZONE_RANGE:8-16_ft.,-4.167882e+12,0.145844,0.140908
SEASON:2009-10,-2.695680e+13,0.420600,0.106977


### 10. Find an optimal value for Elastic Net regression alpha using `ElasticNetCV`.

[Go to the documentation and read how ElasticNetCV works.](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNetCV.html).

Note here that you will be optimizing both the alpha parameter and the l1_ratio:
- `alpha`: strength of regularization
- `l1_ratio`: amount of ridge vs. lasso (0 = all ridge, 1 = all lasso)
    
Do not include 0 in the search for `l1_ratio`: it will not allow it and break!

You can use `n_alphas` for the alpha parameters instead of setting your own values: highly recommended!

Also - be careful setting too many l1_ratios over cross-validation folds in your search. It can take a very long time if you choose too many combinations and for the most part there are diminishing returns in this data.

In [65]:
# A:
elasticnet = ElasticNetCV(l1_ratio=[0.0001, 0.001, 0.01, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
                     alphas=np.logspace(-5, 5, 100), cv=kf)
elasticnet.fit(X, y)
print(elasticnet.alpha_)
print(elasticnet.l1_ratio_)

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 21.352790151265253, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 36.454910939404726, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 159.21895499807306, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Object

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1760.5282491614405, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1729.5067627484495, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1702.6617276286215, tolerance: 1.6974436519258205
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Object

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2588.879956373751, tolerance: 1.7133584165477886
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2550.5734735649053, tolerance: 1.7133584165477886
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2504.681743905231, tolerance: 1.7133584165477886
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objectiv

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1601.8729438782966, tolerance: 1.7133584165477886
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9.245970416976888, tolerance: 1.669793152639087
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 3.4493749710673, tolerance: 1.669793152639087
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective di

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1737.9940474694085, tolerance: 1.669793152639087
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1704.8988298623274, tolerance: 1.669793152639087
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1676.0522483646396, tolerance: 1.669793152639087
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective

...

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 116.04515551098348, tolerance: 1.6754950784593436
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 61.31990067111565, tolerance: 1.6754950784593436
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 35.25995396670987, tolerance: 1.6754950784593436
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objectiv

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 781.3132894661906, tolerance: 1.701202068473609
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 552.059257909646, tolerance: 1.701202068473609
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 551.0433514911601, tolerance: 1.701202068473609
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did

/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 227.52772125347974, tolerance: 1.6767180327868854
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 187.61980584217235, tolerance: 1.6767180327868854
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 295.6971514082802, tolerance: 1.6767180327868854
  tol, rng, random, positive)
/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objecti

0.005336699231206307
1.0


/Users/Noah/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 28.052173232461428, tolerance: 1.8760434531450578
  positive)


### 11. Cross-validate the ElasticNet $R^2$ with the optimal alpha and l1_ratio.

How does it compare to the Ridge and Lasso regularized regressions?

In [12]:
# A:

### 12. [Bonus] Compare the residuals for the Ridge and Lasso visually.


In [13]:
# A: Maybe a histogram?